In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%load_ext cython

In [ ]:
import ROOT
import h5py
import os
from matplotlib import pyplot as plt
import numpy as np
import time
import sys
import datetime
import time

ROOT.gSystem.Load("$MARSSYS/libmars.so")

In [ ]:
filepath_in = "/remote/ceph/user/m/mhuetten/MAGIC_data/merpp/"
filepath_out = "/remote/ceph/user/m/mhuetten/MAGIC_data/ctapipe/"

filename_M1 = "20180821_M1_05074767.014_D_CrabNebula-W0.40+035.root"
filename_M2 = "20180821_M2_05074767.014_D_CrabNebula-W0.40+035.root"

filenames = [filename_M1, filename_M2]
filepaths = [os.path.join(filepath_in, filename_M1), os.path.join(filepath_in, filename_M2)]

infile_M1 = ROOT.TFile(os.path.join(filepath_in, filename_M1))
infile_M2 = ROOT.TFile(os.path.join(filepath_in, filename_M2))

infiles = [infile_M1, infile_M2]

filename_out = ''.join(filename_M1.split("_M1"))

In [ ]:
#ROOT.TBrowser()

In [ ]:
%%cython

#from numba import jit, autojit
#@autojit

def ugly_loop(iter):
   
    cdef int i_pixel, nsample, pixid, fadc_val

    arr_2D = []
    arr_pixid = []
    
    for i_pixel in range(1184):
        arr_pixel = []
        iter.Jump(i_pixel)

        if iter.Jump(i_pixel):
            pixid = iter.GetPixelId()
            arr_pixid.append(pixid)            
            for nsample in range(iter.GetNumHiGainSamples()):
                fadc_val = iter.GetHiGainSampleVal(nsample)
                arr_pixel.append(fadc_val)
            arr_2D.append(arr_pixel)
    return arr_2D, arr_pixid

In [ ]:
def get_ROOT_array(file, tree, branch, leaf):
    tfile = ROOT.TFile(file)
    ttree = getattr(tfile, tree)
    hist = ROOT.TH1D("hist","hist",100,-100,100)
    ttree.Project("hist", "%s.%s"%(branch,leaf),"","",100000000000,0)
    treeArr = ttree.GetV1()
    result = np.array(np.frombuffer(treeArr,count=int(hist.GetEntries())), dtype = np.double)
    return result

In [ ]:
f = h5py.File(os.path.join(filepath_out,filename_out.split(".root")[0] + ".hdf5"),"w")
dsets_events = []

for ntel in range(2):
    
    Header = ROOT.TChain("RunHeaders")
    Header.Add(os.path.join(filepath_in, filenames[ntel]))

    head = ROOT.MRawRunHeader()
    Header.SetBranchAddress("MRawRunHeader.",head)

    # extract global information out of first event:
    Header.GetEntry(0)
    TelescopeNumber =  head.GetTelescopeNumber()
    nevents = head.GetNumEvents()
    #nevents = 2000
    #max_event = Events.GetEntries()
    nevents_check = 0
    
    RunNumber = head.GetRunNumber()
    RunType = head.GetRunTypeStr()
    if RunType == "Data":
        is_mc = False
    elif RunType == "Monte Carlo":
        is_mc = True
        Core_x_vec = get_ROOT_array(filepaths[ntel], "Events", "MMcEvt", "fCoreX")
        Core_y_vec = get_ROOT_array(filepaths[ntel], "Events", "MMcEvt", "fCoreY")
        h_first_vec = get_ROOT_array(filepaths[ntel], "Events", "MMcEvt", "fZFirstInteraction")
    else:
        is_mc = False
    
    Events = ROOT.TChain("Events")
    Events.Add(os.path.join(filepath_in, filenames[ntel]))

    arr_3D = []
    arr_pixid = []
    arr_triggerid = []

    DAQEvtNumber = []
    ClockCounter = []
    NumTrigLvl2 = []
    StereoEvtNumber = []
    TriggerID = []
    TrigPattern = []
    CalibrationPattern = []
    MacrocellsPattern = []
    TimeDiff = []
    if is_mc == True:
        Energy = [] # TeV
        Core_params = [] # m
        X_max = [] # m
        H_first_int = [] # m
        Shower_angles = []

    evtdata = ROOT.MRawEvtData()
    evtheader = ROOT.MRawEvtHeader()
    if is_mc == True:
        mcevtheader = ROOT.MMcEvt()
    
    Events.SetBranchAddress("MRawEvtData.",evtdata)
    Events.SetBranchAddress("MRawEvtHeader.",evtheader)
    if is_mc == True:
        Events.SetBranchAddress("MMcEvt.",mcevtheader)
    
    TrigPatternNames = []
    # valid for runs after 2013-12-13, see MARS MTriggerPatternDecode.cc, ll. 224        
    TrigPatternNames.append('L3 trigger')
    TrigPatternNames.append('Calibration')
    TrigPatternNames.append('LIDAR')
    TrigPatternNames.append('Pedestal')
    TrigPatternNames.append('Pulse injection')
    TrigPatternNames.append('Sumtrigger')
    TrigPatternNames.append('Topo trigger')
    TrigPatternNames.append('L1 trigger')
    
    
    start_time = time.time()

    for nevent in range(nevents):

        if nevent >= nevents:
           continue

        sys.stdout.write("\r ... processing event " + str(nevent) + ", " \
                         + "%s >>> Percent Done : %.2f%%" % (datetime.datetime.now(), 100*nevent/float(nevents)))

        Events.GetEvent(nevent)
        
        trigid = evtheader.GetTriggerID()
        
        binarypattern = '\t{0:016b}'.format(trigid&0xff) 
        
        triggpattern = (bool(int(binarypattern[-8])), 
                        bool(int(binarypattern[-1])), 
                        bool(int(binarypattern[-2])), 
                        bool(int(binarypattern[-3])), 
                        bool(int(binarypattern[-4])), 
                        bool(int(binarypattern[-5])), 
                        bool(int(binarypattern[-6])), 
                        bool(int(binarypattern[-7])))
                
        # check mono-character of run:
        #if nevent > 0:
         #   if triggpattern[0] == False:
         #       if StereoEvtNumber[-1] == StereoEvtNumber[-2]:
         #          StereoEvtNumber[-1] = -1
                #else:
                   # print "calib!"
        
        npixels = evtdata.GetNumPixels()
        
        saturationval = evtdata.GetSaturationVal()
        name = evtdata.GetUniqueName()
        size = evtdata.Sizeof()
        
        iter = ROOT.MRawEvtPixelIter(evtdata)
        
        arr_2D, arr_pixid = ugly_loop(iter)
        
        if arr_2D != []:
            arr_2D = np.array(arr_2D)
            arr_3D.append(arr_2D)
            
            arr_triggerid.append(evtheader.GetTriggerID())
            DAQEvtNumber.append(evtheader.GetDAQEvtNumber())
            ClockCounter.append(evtheader.GetClockCounter())
            NumTrigLvl2.append(evtheader.GetNumTrigLvl2())
            StereoEvtNumber.append(evtheader.GetStereoEvtNumber())
            TriggerID.append(trigid)
            TrigPattern.append(triggpattern)
            CalibrationPattern.append(evtheader.GetCalibrationPattern())
            MacrocellsPattern.append(evtheader.GetMacrocellsPattern())
            TimeDiff.append(evtheader.GetTimeDiff())
            if is_mc == True:
                energy_val = str(mcevtheader.GetEnergyStr())[:-3]
                energy_unit = str(mcevtheader.GetEnergyStr())[-3:]
                if energy_unit == "TeV":
                    Energy.append(float(energy_val))
                elif energy_unit == "GeV":
                    Energy.append(float(energy_val) / 1000.)
                else:
                    raise IOError("Error!")
                Core_params.append((Core_x_vec[nevent] / 100., Core_y_vec[nevent] / 100.)) # meters
                H_first_int.append(h_first_vec[nevent] / 100.) # meters
                    
              
            nevents_check += 1
        #print nevent, arr_2D
    
    end_time = time.time()
    elapsed = end_time - start_time
    print "\n File read into python arrays >>> Elapsed time: %.1f s" % elapsed
    
    ##########################
    # write data to hdf5 file:
    
    arr_triggerid = np.array(arr_triggerid, dtype=np.int16) 
    arr_pixid = np.array(arr_pixid, dtype=np.int16)
    arr_3D = np.array(arr_3D, dtype=np.int16)
    
    dset_name = "MAGIC" + str(ntel+1) + "_rawdata"
    
    dsets_events.append(f.create_dataset(os.path.join(dset_name,"Events"), data=arr_3D.transpose(), compression='gzip'))#, compression_opts=9)
    
    dsets_runheader = []
    dsets_runheader.append(f.create_dataset(os.path.join(dset_name,"RunHeader/PixelIDs"), data=arr_pixid))
    
    f.attrs['RunType'] = RunType
    if nevents_check != nevents:
        print("WARNING: Only "+ str(nevents_check) + " out of supposingly " + str(nevents) + " events filled.")
    f.attrs['NEvents'] = nevents_check
    f.attrs['RunNumber'] = RunNumber
    
    f[dset_name].attrs['TelescopeNumber'] = TelescopeNumber
    
    dsets_evtheader = []
                           
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/DAQEvtNumber"), data=DAQEvtNumber, dtype=np.int16))
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/ClockCounter"), data=ClockCounter))
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/NumTrigLvl2"), data=NumTrigLvl2))
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/StereoEvtNumber"), data=StereoEvtNumber))
    
    #comp_type = np.dtype([('Orbit', 'i'), ('Location', np.str_, 6), ('Temperature (F)', 'f8'), ('Pressure (inHg)', 'f8')])
    
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/TriggerID"), (nevents_check,), np.dtype([('Trigger ID', np.int16)])))
    dsets_evtheader[-1][...] = TriggerID
    
    dt = np.dtype([(TrigPatternNames[0], np.bool), (TrigPatternNames[1], np.bool), (TrigPatternNames[2], np.bool), 
                   (TrigPatternNames[3], np.bool), (TrigPatternNames[4], np.bool), (TrigPatternNames[5], np.bool), 
                   (TrigPatternNames[6], np.bool), (TrigPatternNames[7], np.bool)])
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/TrigPattern"), (nevents_check,), dt))
    dsets_evtheader[-1][...] = np.array(TrigPattern, dtype = dt)
    
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/CalibrationPattern"), data=CalibrationPattern))
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/MacrocellsPattern"), data=MacrocellsPattern))
    dsets_evtheader.append(f.create_dataset(os.path.join(dset_name,"EvtHeader/TimeDiff"), data=TimeDiff))
    
    dsets_events[ntel].attrs['Dimensions'] = ['Number of pixels (increasing pixel ID)', 'Number of samples', 'Number of events']

    # write MC information:
    
    if is_mc == True:
        dsets_mcheader = []
        
        dt = np.dtype([("Energy", np.double),])
        dsets_mcheader.append(f.create_dataset(os.path.join(dset_name,"McHeader/Energy"), (nevents_check,), dt))
        dsets_mcheader[-1][...] = np.array(Energy, dtype = dt)
        dsets_mcheader[-1].attrs['FIELD_0_NAME'] = "Energy"
        dsets_mcheader[-1].attrs['FIELD_0_UNIT'] = "TeV"
        
        dt = np.dtype([("Core_x", np.double), ("Core_y", np.double)])
        dsets_mcheader.append(f.create_dataset(os.path.join(dset_name,"McHeader/Core_xy"), (nevents_check,), dt))
        dsets_mcheader[-1][...] = np.array(Core_params, dtype = dt)
        dsets_mcheader[-1].attrs['FIELD_0_NAME'] = "Core_x"
        dsets_mcheader[-1].attrs['FIELD_1_NAME'] = "Core_y"
        dsets_mcheader[-1].attrs['FIELD_0_UNIT'] = "m"
        dsets_mcheader[-1].attrs['FIELD_1_UNIT'] = "m"

        dt = np.dtype([("H_first_int", np.double),])
        dsets_mcheader.append(f.create_dataset(os.path.join(dset_name,"McHeader/H_first_int"), (nevents_check,), dt))
        dsets_mcheader[-1][...] = np.array(H_first_int, dtype = dt)
        dsets_mcheader[-1].attrs['FIELD_0_NAME'] = "H_first_int"
        dsets_mcheader[-1].attrs['FIELD_0_UNIT'] = "m"
        
    
f.close()

In [ ]:
f.close()